In [3]:
import k3d
import os
import vtk
import numpy as np
from k3d.helpers import download
from vtk.util import numpy_support
import pyvista as pv
import math


def vtm_to_Poly (name):
    filename = name
    reader = vtk.vtkXMLMultiBlockDataReader()
    reader.SetFileName(filename)
    reader.Update()
    
    geometryFilter = vtk.vtkGeometryFilter()
    geometryFilter.SetInputData(reader.GetOutput().GetBlock(0).GetBlock(0))
    geometryFilter.Update()
    return geometryFilter.GetOutput()
    
# read the data
grid = pv.read('mesh/surf0.stl')
grid2 = pv.read('mesh/surf10.stl')


c = vtm_to_Poly ('mesh/block0.vtm')
c2 = vtm_to_Poly ('mesh/block1.vtm')



colors = [0xFF0000,0xaaaaaa,0xFF0000]
grids = [grid,grid,grid2]
conturs = [c,c,c2]
cl_at = ('T', 0.0, 2000)
cl_at1 = ('T', 0.0, 2000)
cl_at2 = ('T', 0.0, 2000)

ra = [cl_at1[1:3],cl_at[1:3],cl_at2[1:3]]


plot = k3d.plot()

plt_vtk = k3d.vtk_poly_data(c ,color_attribute = cl_at, color_map = k3d.basic_color_maps.Jet)
plt_mesh = k3d.vtk_poly_data(grid2, opacity=0.05, wireframe=True,color = 0xaaaaaa)

plot += plt_mesh
plot += plt_vtk


#c_color_range = {str(t):ra[t] for t in range(len(conturs))}
attribute = {str(t):numpy_support.vtk_to_numpy(conturs[t].GetPointData().GetArray(cl_at[0])) for t in range(len(conturs))}
vertices = {str(t):numpy_support.vtk_to_numpy(grids[t].GetPoints().GetData()).astype(np.float32) for t in range(len(grids))}
indices = {str(t):numpy_support.vtk_to_numpy(grids[t].GetPolys().GetData()).reshape(-1, 4)[:, 1:4].astype(np.float32) for t in range(len(grids))}

#print (attribute)


plt_mesh.indices,plt_mesh.vertices = indices,vertices

plt_vtk.attribute = attribute 
plot.fps = 1  # This is important if not set to one then it will interpolate the mesh

plot.display()


Output()

In [6]:
f = open('./volume.html', 'w', encoding='UTF-8')
f.write(plot.get_snapshot(9, 'K3DInstance.startAutoPlay();'))
f.close()